In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/ECE_285_proj/
%cd DeepZip_Compression

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ECE_285_proj
/content/drive/My Drive/ECE_285_proj/DeepZip_Compression


In [2]:
!ls

arithmetic_coder.ipynb	DeepZip_Compression  main-checkpoint.ipynb  __pycache__
data_preparation.py	input.txt	     my_model.h5	    README.md


In [1]:
import sys
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM, Flatten, Conv1D, LocallyConnected1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from math import sqrt
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from matplotlib import pyplot
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
import numpy as np
import argparse
import os
from keras.callbacks import CSVLogger
import keras.backend as K
np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


## DATA PREP

In [2]:
import data_preparation
#Convert letters to integers
integer_encoded, params = data_preparation.process_data('input.txt')

15072434
{'A': 0, 'T': 1, 'C': 2, 'G': 3}
{0: 'A', 1: 'T', 2: 'C', 3: 'G'}
[[3]
 [2]
 [2]
 [1]
 [0]
 [0]
 [3]
 [2]
 [2]
 [1]]
GCCTAAGCCT


In [3]:
bs=128
time_steps=64
num_epochs=20

In [4]:
#Generate traning data and its lable 
X,Y_raw, Y = data_preparation.generate_single_output_data(integer_encoded[: bs * 350  ],bs, time_steps)


(44800, 1)
(44800,)
44672
(44672, 65)


C:\Users\quoca\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Trainning

In [5]:
def custom_loss(y_true, y_pred):
        return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)        

In [6]:
alphabet_size = Y.shape[1]
DZ_model = Sequential()
DZ_model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(Flatten())
DZ_model.add(Dense(64, activation='relu'))
DZ_model.add(Dense(alphabet_size, activation='softmax'))
optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
DZ_model.compile(loss=custom_loss, optimizer=optim)
early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
callbacks_list = [early_stopping]
DZ_model.fit(X, Y, epochs=num_epochs, batch_size=bs, verbose=1, shuffle=True, callbacks=callbacks_list)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
  768/44672 [..............................] - ETA: 2:00 - loss: 1.9583

KeyboardInterrupt: 

## Compression

In [6]:
DZ_model = load_model('my_model.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
class ArithmeticEncoder(object):
    def __init__(self, bitlen):
        self.bit_prec = bitlen   #bit precision 
        self.max_range = 1 << self.bit_prec  #max range based on bit precision 2^bit_prec
        self.mask = self.max_range - 1  #max range index starting at 0
        self.renorm= self.max_range >> 1  #renormalization threshold
        self.second_mask = self.max_range >> 1
        self.low = 0  #initial low
        self.high = self.mask  #initial high
        self.s = 0  

    def update(self, sym, c):
        low = self.low   
        high = self.high
        range = high - low + 1
#         print(sym)
#         print(sym.shape)
        sym = int(sym)
        total = (c[-1])  #cumulative propabilities
        symlow = (c[sym])  
        symhigh = (c[sym+1] )
        

        newlow = low + symlow*range // total  #low in arithmetic integer
        newhigh = low + symhigh*range // total -1 #high in arithemtic integer
        self.low = int(newlow)
        self.high = int(newhigh)
        range = self.high - self.low 
#         print("sym:", sym)
#         print("symlow:", self.low)
#         print("symhigh:", self.high)
#         print("range:", range)
        
        #renormalization
        while((self.low ^ self.high) & self.renorm) == 0:
            self.low = (self.low << 1) 
            range = range << 1 | 1
            self.high = self.low + range
            self.s = self.s + 1          
#         print("renorm low:",self.low)
#         print("renorm range:",range)
#         print("renorm high:", self.high)
   
    def write(self, c, sym):
        self.update(c,sym)
        return [self.low, self.s]
        
    def finish(self):
        self.output.write(1)


# In[25]:


class ArithmeticDecoder(ArithmeticEncoder):
    def __init__(self,statesize,bitin,c):
        self.input = bitin
        self.bitstream = ([int(d) for d in str(self.input)])
        self.code = 0
        self.max_range = 1 << statesize
        self.renorm= self.max_range >> 1
        self.mask = self.max_range - 1
        self.stream = self.input[0:statesize]
        self.low = int(self.stream,2)
        self.t = statesize
        self.thresh = []
        self.total = (c[-1])
        self.cum = [c[i]/self.total for i in range(len(c))]
        self.thresh = [round((self.cum[i])*self.mask) for i in range(len(self.cum))]
        
    def decode(self, c):
#         print("low:",self.low
        c = [c[i]/self.total for i in range(len(c))]
#         print("cum::", c)
        for i in range(len(self.thresh)-1):
            if((self.low < self.thresh[i+1]) & (self.low >= self.thresh[i])):
                sym = i
#                 print("encoded value:",self.low)
#                 print("threshold:", self.thresh)
#                 print("decoded value:", sym)
                rangenew = int(self.thresh[i+1] - self.thresh[i])
#                 print("range:", rangenew)
#                 print("low:", self.thresh[i])
                while (rangenew < self.renorm):
                    rangenew = rangenew << 1 
                    self.low = self.low << 1
                    self.thresh[i] = int(self.thresh[i]) << 1
#                     print("renorm val;",self.low)
#                     print("renorm range;",rangenew)
                    self.thresh[i+1] = rangenew + self.thresh[i]
#                     print("renorm low:" ,self.thresh[i])
                self.thresh = [(c[j]*(self.thresh[i+1]-self.thresh[i]))+self.thresh[i] for j in range(len(self.cum))]
                break
        return sym

In [8]:
alphabet_size = 4
def cumul_d(prob):
    return np.cumsum(prob, axis = 1)  


In [18]:
bitprecision = 64
l = int(X.shape[0] / bs) * bs 
print(l)
print((X.shape[0])-time_steps)

#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.int64)
c[1:] = np.cumsum(prob*10000000 + 1)        
bitstream = []
enc = ArithmeticEncoder(bitprecision)

num_iters = int((len(X)+time_steps)/bs)
ind = np.array(range(bs))*num_iters
prob = DZ_model.predict(X[ind,:], batch_size=bs)
count =0 


cumul_array = np.zeros((X.shape[0], 5))
#Encoding
count = 0
new_l = []
for i in (range(bs)):
    for j in range(min(time_steps, num_iters)):
        if (count%16 == 0 and count != 0):
            enc = None
            enc = ArithmeticEncoder(bitprecision)
            new_l.append(new)
        new = enc.write(X[ind[i], j], c.tolist())
        count = count + 1
new_l.append(new)

count = 0
cumul_array = np.zeros(( round((num_iters - time_steps) * bs), 5), dtype = np.uint64)
new_l1 = []
enc1 = ArithmeticEncoder(bitprecision)
#convert low to bitstream, ensure proper number of bits 
cumul = np.zeros((bs, alphabet_size + 1), dtype = np.uint64)
t = 0
for k in (range(num_iters - time_steps)):
    prob = DZ_model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)
   
    for i in range(bs):
        enc1 = None
        enc1 = ArithmeticEncoder(bitprecision)  
        cumul_array[count, :] = cumul[i,:]
        new = enc1.write(Y_raw[ind[i]], cumul[i,:].tolist())
        new_l1.append(new) 
        count = count + 1
        
    ind = ind + 1







44672
44608


In [19]:
len(new_l1)


36480

In [20]:
cumul_array.shape

(36480, 5)

In [23]:
#Decoding first 64 uniform bits
symdec = []
count = 1
for i in range(len(new_l)):
    count = count + 1
    new = new_l[i]
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream
    #START DECODER
    dec = None
    dec = ArithmeticDecoder(bitprecision, bitstream, c.tolist())
    for i in range(16):
        symdec.append(dec.decode(c.tolist()))   


print("decoded num stream:", symdec)
print(len(symdec))

decoded num stream: [3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 0, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 1, 0, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 0, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 1, 0, 2, 1, 0, 0, 3, 2, 0, 2, 1, 2, 0, 1, 2, 1, 3, 0, 0, 0, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 3, 2, 0, 2, 1, 1, 1, 3, 1, 3, 2, 0, 3, 0, 0, 1, 1, 2, 1, 1, 3, 0, 1, 1, 2, 1, 1, 3, 0, 1, 0, 1, 0, 1, 3, 0, 2, 1, 1, 1, 0, 3, 0, 2, 1, 1, 3, 0, 0, 0, 1, 1, 3, 3, 2, 1, 0, 1, 1, 0, 0, 1, 3, 2, 2, 0, 0, 1, 1, 1, 2, 0, 1, 3, 0, 1, 0, 1, 2, 1, 0, 3, 2, 2, 0, 2, 1, 1, 1, 0, 3, 1, 0, 3, 3, 2, 1, 3, 0, 2

In [24]:
X[0,:]

array([3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1,
       0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2,
       2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1])

In [25]:
#Decode the next (num_iters - time_steps) * bs
symdec = []
new = new_l1[0]
for i in range(len(new_l1)):
    new = new_l1[i]
    c = cumul_array[i, :]
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream
    #START DECODER
    dec = None
    dec = ArithmeticDecoder(bitprecision, bitstream, c.tolist())
    symdec.append(dec.decode(c.tolist()))   
    count = count + 1

print("decoded num stream:", symdec)
print(len(symdec))

decoded num stream: [0, 0, 3, 0, 1, 3, 0, 0, 2, 1, 3, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 3, 0, 1, 1, 2, 0, 0, 2, 1, 2, 0, 2, 0, 0, 3, 2, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0, 1, 2, 1, 1, 0, 0, 3, 2, 2, 1, 2, 1, 1, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 1, 3, 1, 2, 0, 2, 0, 1, 0, 0, 1, 0, 3, 0, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 1, 3, 1, 0, 3, 0, 0, 0, 2, 3, 2, 1, 1, 3, 2, 1, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1, 2, 3, 0, 1, 2, 0, 0, 1, 2, 1, 2, 0, 1, 2, 1, 0, 1, 2, 1, 1, 1, 3, 0, 3, 0, 2, 3, 2, 2, 0, 1, 0, 0, 1, 0, 1, 2, 0, 1, 0, 1, 0, 2, 1, 1, 0, 3, 1, 1, 0, 1, 0, 1, 1, 3, 3, 2, 0, 2, 3, 1, 0, 0, 1, 0, 0, 3, 1, 0, 1, 2, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 1, 2, 1, 2, 2, 0, 1, 2, 0, 0, 2, 0, 3, 0, 1, 2, 2, 1, 1, 3, 0, 0, 0, 2, 2, 1, 0, 0, 0, 1, 0, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 2, 0, 2, 1, 0, 3, 2, 2, 2, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 0, 0

In [26]:
Y_raw.T

array([[0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2,
        2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0,
        3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1,
        0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2,
        2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0,
        3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1,
        0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2,
        2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0,
        3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1,
        0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2,
        2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0,
        3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1,
        0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 3, 2,
        2, 1, 0, 0, 3, 2, 2, 1, 0, 0, 

In [24]:

symdec = []
new = new_l[0]
count = 0
for i in range(len(new_l)):
    new = new_l[i]
    count = count + 1
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream
    #START DECODER
    dec = None
    dec = ArithmeticDecoder(bitprecision, bitstream, cumul_array[count,:])
    for j in range(16):
        symdec.append(dec.decode(c.tolist()))  
        print(symdec)
    count = count + 1

print("decoded num stream:", symdec)
print(len(symdec))
        





[1]
[1, 2]
[1, 2, 1]
[1, 2, 1, 2]
[1, 2, 1, 2, 2]
[1, 2, 1, 2, 2, 2]
[1, 2, 1, 2, 2, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3, 2, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3, 2, 2, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1]
[1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
Y_raw.T

array([[3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1,
        1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3,
        2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0,
        3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1,
        1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3,
        2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0,
        3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1,
        1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3,
        2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0,
        3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1,
        1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3,
        2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0,
        3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 2, 1,
        1, 0, 3, 3, 2, 1, 1, 0, 3, 3, 

In [12]:
symdec = []
new = new_l[0]
count = 0
for i in range(bs):
    for j in range(min(num_iters,timesteps)):
            series_2d[i,j] = dec[i].read(cumul, alphabet_size)


In [24]:
X[0,:]


array([3, 1, 1, 0, 2, 2, 3, 1, 1, 0, 2, 2, 3, 1, 1, 0, 2, 2, 3, 1, 1, 0,
       2, 2, 3, 1, 1, 0, 2, 2, 3, 1, 1, 0, 2, 2, 3, 1, 1, 0, 2, 2, 3, 1,
       1, 0, 2, 2, 3, 1, 1, 0, 2, 2, 3, 1, 1, 0, 2, 2, 3, 1, 1, 0])

In [0]:
 series_2d = np.zeros((bs,num_iters), dtype = np.uint8)
 print(series_2d.shape)

(128, 90)


In [28]:
cumul_array[1,:]

array([       0.,  3096512.,  6913540.,  7035238., 10000004.])

In [0]:
np.asscalar(cumul[-1])


In [0]:
X[i,:].reshape(1,-1).shape

In [0]:
if (l > (len(X.shape[0])-timesteps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    for i in range(bs):
            enc[i].write(cumul[i,:], y_original[ind[i]])
    ind = ind + 1
else:

In [0]:
DZ_model = load_model('my_model.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
#make an array of cumulative probabilities
c = []
for i in range(len(prob)+1):
    c.append(sum(prob[0:i]))

cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)  
for j in (range(num_iters - timesteps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    for i in range(bs):
        new = enc.write(Y_raw[ind[i]], cumul[i,:].tolist())
    ind = ind + 1
    low_final = new[0] #fina low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b')  
while len(bitstream) < (s_final+bitprecision):
    bitstream = '0' + bitstream
    print("encoded bit length:", len(bitstream))


    #START DECODER
    dec = ArithmeticDecoder(bitprecision, bitstream, c)

    symdec = []
    for symbols in range(len(sym)):
        symdec.append(dec.decode())
    print("decoded num stream:", symdec)

    #convert numbers back to letters
    dec_stream = []
    for x, sym in enumerate(symdec): 
        if sym == 0:
            dec_stream.append('a')
        elif sym ==1:
            dec_stream.append('c')
        elif sym == 2:
            dec_stream.append('g')
        elif sym == 3:
            dec_stream.append('t')

    dec_stream = dec_stream[:-1] 
    print("decoded symbol stream:", dec_stream)

NameError: name 'timesteps' is not defined

In [0]:
cumul

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0,

In [0]:

print(X)
print(X[ind,:].shape)

[[0 3 3 ... 3 3 1]
 [3 3 1 ... 3 1 2]
 [3 1 2 ... 1 2 2]
 ...
 [3 0 0 ... 1 1 1]
 [0 0 3 ... 1 1 3]
 [0 3 0 ... 1 3 1]]
(128, 64)


In [0]:
cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)  
print(cumul[0,:].tolist())


[0, 0, 0, 0, 0]


In [0]:
print(sum(prob[1,:]))

1.0000000596046448


In [0]:
import arithmetic_coder

symbol stream input: ['a' 'c' 'g' 't' 't' 'a' 'c' 't' 'g' 'a' 't' 'a' 'a' 't' 'a' 'c' 'c' 'g'
 't']
num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3]
probability: [0.5 0.2 0.2 0.1]
cumulative prob: [0, 0.5, 0.7, 0.8999999999999999, 0.9999999999999999]
encoded bit length: 106
decoded num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3, 1]
decoded symbol stream: ['a', 'c', 'g', 't', 't', 'a', 'c', 't', 'g', 'a', 't', 'a', 'a', 't', 'a', 'c', 'c', 'g', 't']


In [0]:
prob = np.ones(alphabet_size)/alphabet_size
cumul = np.zeros(alphabet_size+1, dtype = float)
cumul[1:] = np.cumsum(prob)        

In [0]:
cumul

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [0]:
prob = np.ones(alphabet_size)/alphabet_size
cumul = np.zeros(alphabet_size+1, dtype = np.uint64)
cumul[1:] = np.cumsum(prob*10000000 + 1)   

In [0]:
cumul

array([       0,  2500001,  5000002,  7500003, 10000004], dtype=uint64)